In [14]:
from routingpy.routers import get_router_by_name
import folium
from shapely.geometry import box, Point
import random

In [15]:
# Create a quick method to generate random coordinates in Berlin

bbox = [13.280066,52.459562,13.507532,52.576611]  # bbox Berlin
minx, miny, maxx, maxy = bbox
poly_berlin = box(*bbox)

def random_coordinates(n, min_dist, max_dist):
    assert min_dist < max_dist # make sure parameters are valid
    
    coordinates = []
    for _ in range(n):
        counter = 0
        in_poly = False
        while not in_poly:
            counter += 1
            x = random.uniform(minx, maxx)
            y = random.uniform(miny, maxy)
            p = Point(x, y)
            if poly_berlin.contains(p):
                # Make sure all route segments are within limits
                if coordinates:
                    if not min_dist < p.distance(Point(coordinates[-1])) < max_dist:
                        continue
                coordinates.append([x, y])
                in_poly = True
            if counter > 1000:
                raise ValueError("Distance settings are too restrictive. Try a wider range and remember it's in degrees.")

    return coordinates

In [16]:
### Create initial map ###

m = folium.Map(location=list(poly_berlin.centroid.coords[0]).reverse())
m.fit_bounds([[miny, minx], [maxy, maxx]])

### Define router parameters ###

# First define the routers and their API key, profile and PolyLine color in the map
routers = {
    'ors': ['your_key', 'driving-car', '#b5152b'],
    'mapbox_osrm': ['your_key', 'driving', '#ff9900'],
    'mapbox_valhalla': ['your_key', 'auto', '#000000'],
    'google': ['your_key', 'driving', '#ff33cc'],
    'graphhopper': ['your_key', 'car', '#417900'],
}

# Then produce the list of coordinates to route from/to and display as-the-crow-flies on the map
route_amount = 2  # define how many random routes you want to calculate
input_coords = [random_coordinates(n=2, min_dist=0.05, max_dist=0.1) for i in range(route_amount)]
for coords in input_coords:
    folium.PolyLine(
        locations=[list(reversed(coord_tuple)) for coord_tuple in coords],
        popup="""Route {}""".format(input_coords.index(coords))
    ).add_to(m)
m

In [17]:
# Finally call each router instance with the same parameters and plot the routes

for router in routers:
    group = folium.FeatureGroup(name=router, show=False)
    api = get_router_by_name(router)(api_key=routers[router][0])
    print("Router: {}".format(router))
    for coords in input_coords:
        print("Location {}".format(input_coords.index(coords)))
        
        # just from A to B without intermediate points
        # distance for 1 degree in Berlin: ~ 110 km latitude, ~68 km longitude, 
        # i.e. 3.4-7 km < distance < 6.8-11 km
        route = api.directions(
            profile=routers[router][1],
            locations=coords
        )
        distance, duration = route.distance / 1000, int(route.duration / 60)
        print("\tDistance: {0:.3f} km\n\tDuration:{1} mins".format(distance, duration))
        popup_html = """
            <h4>Router: {0}</h4><br><br>
            Distance: {1:.3f} km,<br>
            Duration: {2} minutes
        """.format(router, distance, duration)
        folium.PolyLine(
            locations=[list(reversed(coords)) for coords in route.geometry],
            color=routers[router][2],
            weight=3,
            popup=popup_html
        ).add_to(group)
    group.add_to(m)
folium.LayerControl().add_to(m)
m

Router: ors
Location 0
	Distance: 9.223 km
	Duration:16 mins
Location 1
	Distance: 8.480 km
	Duration:12 mins
Router: mapbox_osrm
Location 0
	Distance: 9.150 km
	Duration:15 mins
Location 1
	Distance: 8.819 km
	Duration:10 mins
Router: mapbox_valhalla
Location 0
	Distance: 9.155 km
	Duration:13 mins
Location 1
	Distance: 8.831 km
	Duration:9 mins
Router: google
Location 0
	Distance: 9.105 km
	Duration:15 mins
Location 1
	Distance: 8.817 km
	Duration:9 mins
Router: graphhopper
Location 0
	Distance: 9.068 km
	Duration:14 mins
Location 1
	Distance: 6.717 km
	Duration:6 mins
